In [1]:
import pandas as pd
import numpy as np
from amplpy import AMPL, DataFrame

# Notes

+ Try to cross off time-slots for just one hour, work just with time slots 2+ hours. Ignore less hours.
+ Food recovery shift: Take in recovered food from local bases, and repackage. Designated only for food recover interns. Fluctuate by semester.
+ If mark food recovery, prioritize assigning them here.

+ Priority: make sure specific shifts, morning are filled before later shifts in the day, prioritize restock days. 6-7 on Tuesday and Thursdays. Less people on Fridays and Saturdays.

+ Want every slot in the schedule filled. Minimum for every shift is 1, minimum 2 for last shift of day.

+ Not reschedule every week, usually once per semester

+ Create backup list if there are extra volunteers for certain days/times.

+ Prioritize that each volunteer gets at least one shift, workers who can work more, give more than one shift.

TWO PHASE MODEL - two objectives\
Phase 1\
-minimize shortage cost\
-Cover as much demand as you can

phase 2\
-minimize overfill cost, do we have more workers per placement than we actually need.\
-Can we optimize a little better

# Convert Schedule to DataFrame

# Convert Form Responses to DataFrame

In [2]:
def find_digit(string):
    return [int(s) for s in string.split() if s.isdigit()][0]

In [3]:
useful_columns = np.concatenate((np.array([1]),np.arange(8,25)))
responses = pd.read_csv('FormResults.csv')
responses = responses.iloc[:, useful_columns]
responses.head()

,Username,Which days/times are you available? [7-8am],Which days/times are you available? [8-9am],Which days/times are you available? [9-10am],Which days/times are you available? [10-11am],Which days/times are you available? [11-12pm],Which days/times are you available? [12-1pm],Which days/times are you available? [1-2pm],Which days/times are you available? [2-3pm],Which days/times are you available? [3-4pm],Which days/times are you available? [4-5pm],Which days/times are you available? [5-6pm],Which days/times are you available? [6-7pm ],Which days/times are you available? [7-8pm],Which days/times are you available? [Deliveries only (morning)],Which days/times are you available? [Deliveries only (afternoon)],Which days/times are you available? [Food Recovery only],Approximately how many hours per week are you available to volunteer?
0,jake@berkeley.edu,Monday;Tuesday;Saturday,Tuesday;Saturday,Monday;Tuesday;Saturday,Tuesday;Thursday,Tuesday;Thursday,Monday;Tuesday;Thursday,Tuesday;Thursday,Tuesday;Wednesday;Friday,Tuesday;Wednesday;Friday,Tuesday;Wednesday;Friday,Monday;Tuesday,Tuesday,Monday;Tuesday,Tuesday,Monday;Tuesday,Tuesday,4 hours / week


In [4]:
def response_tranform(df):
    abbvs = ['m', 't', 'w', 'th', 'f', 's']
    times = ['7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', 'mornDeliv', 'afterDeliv', 'foodRecov']
    columns = ['User', 'hours']
    for i in times:
        for j in abbvs:
            columns.append(j + '-' + i)
    df_transform = pd.DataFrame(columns=columns)
    for i in np.arange(df.shape[0]):
        data = []
        for j in df.iloc[i, 1:17]:
            if 'Monday' in j:
                data.append(1)
            else:
                data.append(0)
            if 'Tuesday' in j:
                data.append(1)
            else:
                data.append(0)
            if 'Wednesday' in j:
                data.append(1)
            else:
                data.append(0)
            if 'Thursday' in j:
                data.append(1)
            else:
                data.append(0)
            if 'Friday' in j:
                data.append(1)
            else:
                data.append(0)
            if 'Saturday' in j:
                data.append(1)
            else:
                data.append(0)
        data = [df.iloc[i, 0]] + [df.iloc[i, 17]] + data
        df_length = len(df_transform)
        df_transform.loc[df_length] = data
    return df_transform

In [5]:
response_transform = response_tranform(responses)

In [6]:
response_transform['hours'] = response_transform['hours'].apply(find_digit)

In [7]:
response_transform.head()

,User,hours,m-7,t-7,w-7,th-7,f-7,s-7,m-8,t-8,...,w-afterDeliv,th-afterDeliv,f-afterDeliv,s-afterDeliv,m-foodRecov,t-foodRecov,w-foodRecov,th-foodRecov,f-foodRecov,s-foodRecov
0,jake@berkeley.edu,4,1,1,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0


## A new transform - availability

Columns:
+ User
+ Day
+ Time
+ Hours want to volunteer - hourVol
+ 

In [8]:
availability = response_transform.copy()
def availability_transform(df):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-8-4742441cac30>, line 3)

# Run Constraint Satisfaction Algorithm

Variables: user-day-shift
Constraints:
+ 1 <= sum(day-shift) <= 5
+ 1 <= sum(user) <= user.ourVol
+ sum(user-mon-7-8) <= 5

In [9]:
#pip install python-constraint
from constraint import *

In [10]:
#problem = Problem()
#problem.addVariables(["a", "b"], [1, 2, 3])
#problem.addConstraint(lambda a, b: b == a+1, ["a", "b"])
#solutions = problem.getSolutions()

# Demo

In [11]:
shifts = [1, 2, 3, 4]
users = ['A', 'B', 'C', 'D', 'E', 'F']
num_hours = [1, 3, 2, 2, 2, 1]
variable_set = [0, 1]

variables = []
for i in shifts:
    for j in users:
        variables.append(j + '-' + str(i))

In [12]:
variables

['A-1',
 'B-1',
 'C-1',
 'D-1',
 'E-1',
 'F-1',
 'A-2',
 'B-2',
 'C-2',
 'D-2',
 'E-2',
 'F-2',
 'A-3',
 'B-3',
 'C-3',
 'D-3',
 'E-3',
 'F-3',
 'A-4',
 'B-4',
 'C-4',
 'D-4',
 'E-4',
 'F-4']

In [13]:
my = ("B-1", "B-2", "B-3", "B-4")

Takes a while to run>

In [26]:
def bool_func(*arg):
    return sum(arg) == 1

In [27]:
def leeqN(x):
    def leeq_helper(*arg):
        return sum(arg) <= x
    return leeq_helper

In [28]:
leeqN(3)(1, 1)

True

In [29]:
problem = Problem()
problem.addVariables(variables, variable_set)
#user constraints based on volunteer hours (volunteer shifts)
#lower bound of 1 on each user (to ensure every volunteer has one shift):
##user A constraint (max 1)
problem.addConstraint(bool_func, ("A-1", "A-2", "A-3", "A-4"))
##user B constraint (max 3)
problem.addConstraint(leeqN(3), ("B-1", "B-2", "B-3", "B-4"))
problem.addConstraint(lambda b_1, b_2, b_3, b_4: b_1 + b_2 + b_3 + b_4 >= 1, ("B-1", "B-2", "B-3", "B-4"))
##user C constraint (max 2)
problem.addConstraint(lambda c_1, c_2, c_3, c_4: c_1 + c_2 + c_3 + c_4 <= 2, ("C-1", "C-2", "C-3", "C-4"))
problem.addConstraint(lambda c_1, c_2, c_3, c_4: c_1 + c_2 + c_3 + c_4 >= 1, ("C-1", "C-2", "C-3", 'C-4'))
##user D constraint (max 2)
problem.addConstraint(lambda d_1, d_2, d_3, d_4: d_1 + d_2 + d_3 + d_4 <= 2, ("D-1", "D-2", "D-3", "D-4"))
problem.addConstraint(lambda d_1, d_2, d_3, d_4: d_1 + d_2 + d_3 + d_4 >= 1, ("D-1", "D-2", "D-3", "D-4"))
##user E constraint (max 2)
problem.addConstraint(lambda e_1, e_2, e_3, e_4: e_1 + e_2 + e_3 + e_4 <= 2, ("E-1", "E-2", "E-3", "E-4"))
problem.addConstraint(lambda e_1, e_2, e_3, e_4: e_1 + e_2 + e_3 + e_4 >= 1, ("E-1", "E-2", "E-3", "E-4"))
##user F constraint (max 1)
problem.addConstraint(bool_func, ("F-1", "F-2", "F-3", "F-4"))


#shift based constraints (filling each shift but not overfilling)
#lower bound (of 1) on shifts
##shift 1 constraints
problem.addConstraint(lambda a_1, b_1, c_1, d_1, e_1, f_1: a_1 + b_1 + c_1 + d_1 + e_1 + f_1 <= 5, ("A-1", "B-1", "C-1", "D-1", "E-1", "F-1"))
problem.addConstraint(lambda a_1, b_1, c_1, d_1, e_1, f_1: a_1 + b_1 + c_1 + d_1 + e_1 + f_1 >= 1, ("A-1", "B-1", "C-1", "D-1", "E-1", "F-1"))
##shift 2 constraints
problem.addConstraint(lambda a_2, b_2, c_2, d_2, e_2, f_2: a_2 + b_2 + c_2 + d_2 + e_2 + f_2 <= 5, ("A-2", "B-2", "C-2", "D-2", "E-2", "F-2"))
problem.addConstraint(lambda a_2, b_2, c_2, d_2, e_2, f_2: a_2 + b_2 + c_2 + d_2 + e_2 + f_2 >= 1, ("A-2", "B-2", "C-2", "D-2", "E-2", "F-2"))
##shift 3 constraints
problem.addConstraint(lambda a_3, b_3, c_3, d_3, e_3, f_3: a_3 + b_3 + c_3 + d_3 + e_3 + f_3 <= 5, ("A-3", "B-3", "C-3", "D-3", "E-3", "F-3"))
problem.addConstraint(lambda a_3, b_3, c_3, d_3, e_3, f_3: a_3 + b_3 + c_3 + d_3 + e_3 + f_3 >= 1, ("A-3", "B-3", "C-3", "D-3", "E-3", "F-3"))
##shift 4 constraints
problem.addConstraint(lambda a_4, b_4, c_4, d_4, e_4, f_4: a_4 + b_4 + c_4 + d_4 + e_4 + f_4 <= 5, ("A-4", "B-4", "C-4", "D-4", "E-4", "F-4"))
problem.addConstraint(lambda a_4, b_4, c_4, d_4, e_4, f_4: a_4 + b_4 + c_4 + d_4 + e_4 + f_4 >= 1, ("A-4", "B-4", "C-4", "D-4", "E-4", "F-4"))

solutions = problem.getSolutions()

In [30]:
basic_df = pd.DataFrame(solutions)
basic_df

,B-1,B-2,B-3,B-4,C-1,C-2,C-3,C-4,D-1,D-2,...,E-3,E-4,A-1,F-1,A-2,A-3,A-4,F-4,F-2,F-3
0,1,1,1,0,1,1,0,0,1,1,...,0,0,1,0,0,0,0,1,0,0
1,1,1,1,0,1,1,0,0,1,1,...,0,0,0,0,1,0,0,1,0,0
2,1,1,1,0,1,1,0,0,1,1,...,0,0,0,0,0,1,0,1,0,0
3,1,1,1,0,1,1,0,0,1,1,...,0,0,0,1,0,0,1,0,0,0
4,1,1,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170819,0,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
170820,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,0,0,0,0,1,0
170821,0,0,0,1,0,0,0,1,0,0,...,1,1,0,1,1,0,0,0,0,0
170822,0,0,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,1,0


## Creating a function to collect relevant data from potential solutions

In [ ]:
def solutions_toDataFrame(sol):
    df = pd.DataFrame(solutions)
    for user in users:
        one_df = df.filter(regex=user)
        df[user + "-total shifts"] = one_df.sum(axis=1)
    for shift in shifts:
        one_df = df.filter(regex=str(shift))
        df["Shift " + str(shift) + "-fill"] = one_df.sum(axis=1)
    return df

In [ ]:
sol_df = solutions_toDataFrame(solutions)

In [ ]:
sol_df

1. First, assign shifts to volunteers with fewest available times.
2. Next, assign remaining shifts (1-shift per person) til all shifts filled.
3. 